In [2]:
# Testing the fine tuned model
import sys
import os
# Add the project root directory to the PYTHONPATH
project_root = os.path.abspath(os.path.join(".."))
print(f"Project root: {project_root}")
sys.path.append(project_root)
from agents.generator import Generator

# model_lora = Generator(
#     base_model_name="mistralai/Mistral-7B-Instruct-v0.2", 
#     adapter_dir="../saved_models/lora_finetuned/20250512-204039/checkpoint-1000", 
#     device="cpu"
# )

base_model = Generator(
    base_model_name="mistralai/Mistral-7B-Instruct-v0.2", 
    device="cpu",
    finetuned=False
)

Project root: /data/amciilab/yiming/LLM/Medical_QA


Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.18it/s]


In [9]:
# prompt = "<s>[INST] QUESTION: What are the symptoms of Alzheimer's disease? \nAnswer: [INST]"
# prompt = "What are the symptoms of Alzheimer's disease?"
prompt = "<s>[INST] Who are you? [INST]"
# response = model_lora.generate(prompt, temperature=0.7, max_length=200, top_p=0.95, top_k=50)
# print(response)

print("=============================================")

response_base = base_model.generate(prompt, temperature=0.7, max_length=200, top_p=0.95, top_k=50)
# response_base = base_model.generate(prompt, max_new_tokens=200, do_sample=False)
print(response_base)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Who are you? [INST] I am an artificial intelligence designed to assist with information and answer questions to the best of my ability. I don't have the ability to have a personality or emotions. I exist to help provide information and answer questions. How may I assist you today?


In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# prompt = "<s>[INST] Who are you? [/INST]"
# prompt = "Who are you?"
prompt = "<s>[INST] What are the symptoms of diabetes? [/INST]"
base_model_name="mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(base_model_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False
)
tokenizer.decode(output[0], skip_special_tokens=False)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.45s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"<s><s>[INST] What are the symptoms of diabetes? [/INST] Diabetes, specifically type 1 and type 2, can present with various symptoms. Here are some common symptoms that may indicate the presence of diabetes:\n\n1. Increased thirst and frequent urination: Excess sugar in the blood can cause an increased need to urinate and an increased thirst as the body tries to flush out the excess sugar through urine.\n\n2. Increased hunger: People with diabetes may experience increased hunger due to the body's in"

In [29]:
prompt = "[INST] What are the symptoms of diabetes? [/INST]"
inputs = tokenizer(prompt, return_tensors="pt")
tokenizer.decode(inputs['input_ids'][0])

'<s> [INST] What are the symptoms of diabetes? [/INST]'

In [ ]:
# Testing the RAG pipeline
from scripts.rag_pipeline import RAGPipeline
pipeline = RAGPipeline(
        retriever_path="../data/rag/rag.json",
        retriever_index_path="../data/rag/pubmedqa_index.faiss",
        context_model_name="mistralai/Mistral-7B-v0.1",
        generator_model_name="mistralai/Mistral-7B-v0.1",
        generator_model_dir="../saved_models/lora_finetuned/20250512-204039/checkpoint-1000",
        fact_checker_model="all-mpnet-base-v2",
        summarizer_model="facebook/bart-large-cnn",
        device="cpu"
)   
query = "What are the symptoms of Alzheimer's disease?"
final_response = pipeline.run(
        query, 
        top_k_ret=3,
        top_k_gen=100,
        top_p_gen=0.95,
        max_length_gen=512
)
print("\nFinal Response:", final_response)

In [2]:
data_zip_path="../data/raw/data_clean.zip"
extract_dir="../data/raw"
import zipfile
def unzip_data(data_zip_path, extract_dir):
    print(f"Extracting {data_zip_path}...")
    with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted to {extract_dir}")
    
unzip_data(data_zip_path, extract_dir)

Extracting ../data/raw/data_clean.zip...
Extracted to ../data/raw
